In [1]:
import sys
from pathlib import Path

notebook_path = Path().resolve()
project_root = notebook_path.parent
sys.path.append(str(project_root))
print('Project root added:', project_root)

Project root added: /Users/arimmean/Desktop/Music_MMLS


In [2]:
import os
import torch
from torch.utils.data import DataLoader
from torch import nn, optim

import wandb

from Music_MMLS.data.dataset import Music_Dataset
from Music_MMLS.models.unet import UNet
from Music_MMLS.training.train import Trainer


/Users/arimmean/Desktop/Music_MMLS/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/arimmean/Desktop/Music_MMLS/.venv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
from hydra import initialize, compose
from omegaconf import OmegaConf

with initialize(config_path='../configs', job_name='demo'):
    cfg = compose(config_name='config')

print('Hydra Config:\n')
print(OmegaConf.to_yaml(cfg))

Hydra Config:

project:
  name: Music_MMLS
  seed: 42
  device: cuda
dataset:
  size: 500
  clean_dir: ../content/sample_data/Data/all_records
  noise_dir: ../content/sample_data/Data/noise
model:
  name: UNet
  n_channels: 1
training:
  epochs: 10
  learning_rate: 0.001
  batch_size: 4
  optimizer: Adam
  loss_fn: MSELoss



/var/folders/_m/krn2rhrs0pj5wb6b32_7bf7h0000gn/T/ipykernel_77827/708244998.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path='../configs', job_name='demo'):


In [4]:
clean_files = [os.path.join(cfg.dataset.clean_dir, f)
               for f in os.listdir(cfg.dataset.clean_dir) if f.endswith('.wav')]
noise_files = [os.path.join(cfg.dataset.noise_dir, f) 
               for f in os.listdir(cfg.dataset.noise_dir) if f.endswith('.wav')]

print(f'Found {len(clean_files)} clean files and {len(noise_files)} noise files.')

Found 999 clean files and 5000 noise files.


In [5]:
batch_size = cfg.training.batch_size
num_epochs = cfg.training.epochs
learning_rate = cfg.training.learning_rate

dataset = Music_Dataset(size=cfg.dataset.size, clean_files=clean_files, noise_files=noise_files)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

device = torch.device(cfg.project.device if torch.cuda.is_available() else 'cpu')

model = UNet(n_channels=cfg.model.n_channels)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    device=device
)

print(f'Training on {device} with batch size {batch_size} for {num_epochs} epochs.')

Training on cpu with batch size 4 for 10 epochs.


In [7]:
history = trainer.fit(
    train_loader=dataloader,
    val_loader=dataloader,
    num_epochs=num_epochs,
    save_best_path="../checkpoints/unet_final"
)

print("Training complete. Final training history:")
print(history)

Train:  10%|▉         | 12/125 [00:08<01:17,  1.47it/s]


KeyboardInterrupt: 